In [44]:
from pathlib import Path
from itertools import islice
import configparser

from sqlalchemy import create_engine
import requests

import numpy as np
import pandas as pd

In [45]:
DATA_PATH = Path('../data')
CONFIG_PATH = Path('../config')

In [48]:
config = configparser.ConfigParser()
config.read(CONFIG_PATH/'test_sources.ini')

['../config/test_sources.ini']

In [49]:
CSV_URL = config['DEFAULT']['CSV_URL']
JSON_URL = config['DEFAULT']['JSON_URL']
SHARED_DB_CON = config['DEFAULT']['SHARED_DB_CON']
PRIVATE_DB_CON = config['DEFAULT']['PRIVATE_DB_CON']

**.csv file**

In [50]:
cols = ['order_id', 'order_uuid', 'good_title', 'date', 'amount', 'name', 'email']

orders_df = pd.read_csv(CSV_URL)
orders_df.columns = cols
orders_df['order_id'] = orders_df['order_id'].astype('str')
orders_df['order_date'] = pd.to_datetime(orders_df['date'])
orders_df.head()

,order_id,order_uuid,good_title,date,amount,name,email,order_date
0,76921,7d9a28f7-418b-4e8a-9ab3-68bf5deba724,Банк триста плод сынок неудобно поезд неожидан...,2020-05-15 8:31:05,3,Елизавета Семеновна Юдина,belovaanna@mail.ru,2020-05-15 08:31:05
1,45388,f70a0b8c-f6cd-4876-b21b-33ee76d2d993,Пропадать беспомощный равнодушный.,2020-06-01 3:08:10,5,Ершов Милий Григорьевич,doroninleon@rao.com,2020-06-01 03:08:10
2,68345,5507582b-992f-4eb0-97e6-6d4b5d66b06c,Что мягкий роса научить необычный домашний ком...,2020-05-12 21:50:55,4,Маслов Твердислав Фадеевич,doroninatatjana@rambler.ru,2020-05-12 21:50:55
3,54184,c48a1e2e-06cf-4cb5-b697-0c695227b450,Товар назначить медицина секунда увеличиваться.,2020-05-29 7:17:52,3,Анисим Харитонович Лазарев,komarovepifan@yahoo.com,2020-05-29 07:17:52
4,67254,1d83bf09-4c3a-458f-91c3-bb27c475ede8,Скрытый наступать предоставить невозможно посв...,2020-05-09 17:07:10,3,Силина Элеонора Филипповна,kirill2008@zao.com,2020-05-09 17:07:10


In [51]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 8 columns):
order_id      895 non-null object
order_uuid    895 non-null object
good_title    895 non-null object
date          895 non-null object
amount        895 non-null int64
name          895 non-null object
email         895 non-null object
order_date    895 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 56.1+ KB


In [52]:
orders_df.describe(include='all')

,order_id,order_uuid,good_title,date,amount,name,email,order_date
count,895,895,895,895,895.000000,895,895,895
unique,893,895,895,895,NaN,895,895,895
top,34183,dbd8068a-bfbf-42c4-9101-726c7e2137df,Намерение покинуть задрать пища господь страст...,2020-05-01 16:13:27,NaN,Филимон Матвеевич Михеев,zuevkarp@ooo.org,2020-05-04 23:50:18
freq,2,1,1,1,NaN,1,1,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-01 00:33:23
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-02 18:03:08
mean,NaN,NaN,NaN,NaN,2.930726,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,1.453109,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN


In [53]:
orders_df['amount'].unique()

array([3, 5, 4, 2, 1])

In [54]:
order_uuids = set(orders_df['order_uuid'])

In [55]:
orders_df.duplicated().sum()

0

In [56]:
orders_df[orders_df['order_id'].duplicated(keep=False)].sort_values('order_id')

,order_id,order_uuid,good_title,date,amount,name,email,order_date
450,22151,93017327-f526-4134-98b3-19c9232d6c76,Художественный житель слишком экзамен идея.,2020-05-15 14:21:32,4,Миронов Радислав Власович,arhipovsilvestr@zao.com,2020-05-15 14:21:32
857,22151,29a4e452-683d-4e33-8d6b-65ac955d0e92,Счастье инвалид роскошный спалить.,2020-05-25 6:41:35,2,Хохлова Алла Васильевна,nekrasovepifan@rambler.ru,2020-05-25 06:41:35
56,34183,1494c7ed-6438-4676-9f45-d16acd72c546,Мальчишка нож строительство отражение дьявол с...,2020-05-18 15:12:38,5,Ия Егоровна Гордеева,kirill1985@yahoo.com,2020-05-18 15:12:38
830,34183,ff6f216b-5f49-48b6-8d07-43121dd685b6,Чем социалистический спорт лиловый перебивать ...,2020-05-12 7:16:58,4,Евсеева Юлия Степановна,turovaanna@ooo.edu,2020-05-12 07:16:58


**.json file**

In [57]:
r = requests.get(JSON_URL)
r.raise_for_status()

status_data = r.json()

In [58]:
list(status_data.items())[:5]

[('7d9a28f7-418b-4e8a-9ab3-68bf5deba724', {'success': True, 'errors': []}),
 ('f70a0b8c-f6cd-4876-b21b-33ee76d2d993',
  {'success': False,
   'errors': ['The order failed because shopper payment details were incorrect or insufficient.',
    'Card unavailable',
    'You entered an invalid card number. Please try again. Please note we only accept VISA MASTERCARD AMEX .']}),
 ('5507582b-992f-4eb0-97e6-6d4b5d66b06c', {'success': True, 'errors': []}),
 ('c48a1e2e-06cf-4cb5-b697-0c695227b450', {'success': True, 'errors': []}),
 ('1d83bf09-4c3a-458f-91c3-bb27c475ede8',
  {'success': False,
   'errors': ['The order failed because shopper payment details were incorrect or insufficient.',
    'This transaction has been declined. Please try a different card or contact the credit card provider for assistance',
    'Card unavailable']})]

In [59]:
for order_uuid, data in islice(status_data.items(), 5):
    print(order_uuid, data.keys(), data['success'], len(data['errors']))

7d9a28f7-418b-4e8a-9ab3-68bf5deba724 dict_keys(['success', 'errors']) True 0
f70a0b8c-f6cd-4876-b21b-33ee76d2d993 dict_keys(['success', 'errors']) False 3
5507582b-992f-4eb0-97e6-6d4b5d66b06c dict_keys(['success', 'errors']) True 0
c48a1e2e-06cf-4cb5-b697-0c695227b450 dict_keys(['success', 'errors']) True 0
1d83bf09-4c3a-458f-91c3-bb27c475ede8 dict_keys(['success', 'errors']) False 3


In [60]:
status_df = pd.DataFrame(status_data.values(), index=status_data.keys()).reset_index()
status_df.columns = ['order_uuid', 'success', 'errors']
status_df.head()

,order_uuid,success,errors
0,7d9a28f7-418b-4e8a-9ab3-68bf5deba724,True,[]
1,f70a0b8c-f6cd-4876-b21b-33ee76d2d993,False,[The order failed because shopper payment deta...
2,5507582b-992f-4eb0-97e6-6d4b5d66b06c,True,[]
3,c48a1e2e-06cf-4cb5-b697-0c695227b450,True,[]
4,1d83bf09-4c3a-458f-91c3-bb27c475ede8,False,[The order failed because shopper payment deta...


In [61]:
status_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 3 columns):
order_uuid    895 non-null object
success       895 non-null bool
errors        895 non-null object
dtypes: bool(1), object(2)
memory usage: 15.0+ KB


In [62]:
status_uuids = set(status_df.index)

In [63]:
len(order_uuids & status_uuids)

0

In [64]:
status_df.reset_index()

,index,order_uuid,success,errors
0,0,7d9a28f7-418b-4e8a-9ab3-68bf5deba724,True,[]
1,1,f70a0b8c-f6cd-4876-b21b-33ee76d2d993,False,[The order failed because shopper payment deta...
2,2,5507582b-992f-4eb0-97e6-6d4b5d66b06c,True,[]
3,3,c48a1e2e-06cf-4cb5-b697-0c695227b450,True,[]
4,4,1d83bf09-4c3a-458f-91c3-bb27c475ede8,False,[The order failed because shopper payment deta...
...,...,...,...,...
890,890,e1225cc6-5f1e-4754-8c6a-d5c40769da1b,False,[Card unavailable]
891,891,746cea7f-0b39-4dd8-8c80-182cc98035d3,True,[]
892,892,1f3b853d-7ab1-4087-b1a1-757a016eeb05,False,"[Operation denied by bank, INVALID_TRANSACTION..."
893,893,789717a6-5e5b-4434-b724-1f974a6e910a,True,[]


**Shared DB**

In [65]:
shared_con = create_engine(SHARED_DB_CONN_URL)

In [66]:
customers_df = pd.read_sql_table('customers', shared_con)
customers_df['id'] = customers_df['id'].astype('str')
customers_df.head()

,id,name,birth_date,gender,email
0,97766,Нинель Васильевна Носова,1975-07-19,F,belovaanna@mail.ru
1,96374,Валерия Борисовна Фомина\t,1962-01-12,F,doroninleon@rao.com
2,38920,Ладислав Аверьянович Щукин,1933-05-16,M,doroninatatjana@rambler.ru
3,62940,Анна Кирилловна Беспалова,2005-08-30,F,komarovepifan@yahoo.com
4,44359,Ирина Сергеевна Кулагина,1913-03-25,F,kirill2008@zao.com


In [67]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 5 columns):
id            889 non-null object
name          889 non-null object
birth_date    889 non-null datetime64[ns]
gender        889 non-null object
email         889 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 34.9+ KB


In [68]:
customers_df.describe(include='all')

,id,name,birth_date,gender,email
count,889,889,889,889,889
unique,889,889,879,2,889
top,84128,Маслов Ярослав Валерьянович\t,1950-03-29 00:00:00,M,zuevkarp@ooo.org
freq,1,1,2,445,1
first,NaN,NaN,1904-06-19 00:00:00,NaN,NaN
last,NaN,NaN,2020-06-03 00:00:00,NaN,NaN


In [69]:
items_df = pd.read_sql_table('goods', shared_con)
items_df.columns = ['id', 'good_title', 'price']
items_df['id'] = items_df['id'].astype('str')
items_df.head()

,id,good_title,price
0,25389,Банк триста плод сынок неудобно поезд неожидан...,1835.98
1,54759,Пропадать беспомощный равнодушный.,375.75
2,56751,Что мягкий роса научить необычный домашний ком...,3354.86
3,78120,Товар назначить медицина секунда увеличиваться.,5270.20
4,85971,Скрытый наступать предоставить невозможно посв...,1765.52


In [70]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 3 columns):
id            895 non-null object
good_title    895 non-null object
price         895 non-null float64
dtypes: float64(1), object(2)
memory usage: 21.1+ KB


In [71]:
items_df.describe(include='all')

,id,good_title,price
count,895,895,895.000000
unique,893,895,NaN
top,37258,Намерение покинуть задрать пища господь страст...,NaN
freq,2,1,NaN
mean,NaN,NaN,4978.378212
std,NaN,NaN,2891.481750
min,NaN,NaN,17.820000
25%,NaN,NaN,2502.935000
50%,NaN,NaN,4889.170000
75%,NaN,NaN,7475.395000


**Prepare_data**

In [72]:
customers_df['age'] = (pd.datetime.now() - customers_df['birth_date']).div(pd.Timedelta(value=1, unit='Y')).astype('int')
customers_df['age'].describe()

/home/jupyter/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: M and Y units are deprecated and will be removed in a future version.
  """Entry point for launching an IPython kernel.


count    889.000000
mean      56.902137
std       34.183996
min        0.000000
25%       27.000000
50%       58.000000
75%       87.000000
max      115.000000
Name: age, dtype: float64

In [73]:
customers_df['name'] = customers_df['name'].str.strip()
customers_df['email'] = customers_df['email'].str.strip()

In [74]:
orders_df['name'] = orders_df['name'].str.strip()
orders_df['email'] = orders_df['email'].str.strip()

In [75]:
status_df['payment_status'] = np.where(status_df['success'], 'success', 'fail')

**Merge data**

In [76]:
orders_cols = ['order_uuid', 'good_title', 'date', 'amount', 'name', 'email']
status_cols = ['order_uuid', 'payment_status']
customers_cols = ['id', 'email', 'age']
items_cols = ['good_title', 'price']

In [77]:
full_df = orders_df[orders_cols].merge(status_df[status_cols], on='order_uuid', right_index=True)
full_df = full_df.merge(customers_df[customers_cols], how='left', on='email')
full_df = full_df.merge(items_df[items_cols], how='left', on='good_title')

full_df['total_price'] = full_df.eval('amount * price')

full_df

,order_id,order_uuid,good_title,date,amount,name,email_x,payment_status,id,email_y,age,price,total_price
0,76921,7d9a28f7-418b-4e8a-9ab3-68bf5deba724,Банк триста плод сынок неудобно поезд неожидан...,2020-05-15 8:31:05,3,Елизавета Семеновна Юдина,belovaanna@mail.ru,success,NaN,NaN,NaN,1835.98,5507.94
1,45388,f70a0b8c-f6cd-4876-b21b-33ee76d2d993,Пропадать беспомощный равнодушный.,2020-06-01 3:08:10,5,Ершов Милий Григорьевич,doroninleon@rao.com,fail,NaN,NaN,NaN,375.75,1878.75
2,68345,5507582b-992f-4eb0-97e6-6d4b5d66b06c,Что мягкий роса научить необычный домашний ком...,2020-05-12 21:50:55,4,Маслов Твердислав Фадеевич,doroninatatjana@rambler.ru,success,NaN,NaN,NaN,3354.86,13419.44
3,54184,c48a1e2e-06cf-4cb5-b697-0c695227b450,Товар назначить медицина секунда увеличиваться.,2020-05-29 7:17:52,3,Анисим Харитонович Лазарев,komarovepifan@yahoo.com,success,NaN,NaN,NaN,5270.20,15810.60
4,67254,1d83bf09-4c3a-458f-91c3-bb27c475ede8,Скрытый наступать предоставить невозможно посв...,2020-05-09 17:07:10,3,Силина Элеонора Филипповна,kirill2008@zao.com,fail,NaN,NaN,NaN,1765.52,5296.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,85355,e1225cc6-5f1e-4754-8c6a-d5c40769da1b,Развернуться поймать ручей палка.,2020-05-23 15:18:44,1,Уваров Валерьян Арсеньевич,ratiborkudrjashov@yandex.ru,fail,NaN,NaN,NaN,9757.25,9757.25
891,21044,746cea7f-0b39-4dd8-8c80-182cc98035d3,Кузнец спасть желание избегать сопровождаться ...,2020-06-01 3:21:56,2,Марфа Геннадиевна Стрелкова,izjaslavkuznetsov@rao.ru,success,NaN,NaN,NaN,2635.40,5270.80
892,67729,1f3b853d-7ab1-4087-b1a1-757a016eeb05,Белье плод нож карандаш металл правильный поси...,2020-05-19 20:21:47,5,г-н Гаврилов Ладимир Дорофеевич,semenovaevdokija@vishnjakov.biz,fail,NaN,NaN,NaN,6956.57,34782.85
893,50744,789717a6-5e5b-4434-b724-1f974a6e910a,Единый человечек о лететь эффект смелый смерте...,2020-05-03 18:00:49,5,Евсеев Юлий Владиславович,trifonstrelkov@gmail.com,success,NaN,NaN,NaN,5308.94,26544.70


In [78]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 895 entries, 0 to 894
Data columns (total 13 columns):
order_id          895 non-null object
order_uuid        895 non-null object
good_title        895 non-null object
date              895 non-null object
amount            895 non-null int64
name              895 non-null object
email_x           895 non-null object
payment_status    895 non-null object
id                20 non-null object
email_y           20 non-null object
age               20 non-null float64
price             895 non-null float64
total_price       895 non-null float64
dtypes: float64(3), int64(1), object(9)
memory usage: 97.9+ KB


In [79]:
cols = ['name', 'age', 'good_title', 'date', 'payment_status', 'total_price', 'amount']

full_df[cols].head()

,name,age,good_title,date,payment_status,total_price,amount
0,Елизавета Семеновна Юдина,NaN,Банк триста плод сынок неудобно поезд неожидан...,2020-05-15 8:31:05,success,5507.94,3
1,Ершов Милий Григорьевич,NaN,Пропадать беспомощный равнодушный.,2020-06-01 3:08:10,fail,1878.75,5
2,Маслов Твердислав Фадеевич,NaN,Что мягкий роса научить необычный домашний ком...,2020-05-12 21:50:55,success,13419.44,4
3,Анисим Харитонович Лазарев,NaN,Товар назначить медицина секунда увеличиваться.,2020-05-29 7:17:52,success,15810.60,3
4,Силина Элеонора Филипповна,NaN,Скрытый наступать предоставить невозможно посв...,2020-05-09 17:07:10,fail,5296.56,3


**Private DB**